<a href="https://colab.research.google.com/github/AhamedShimak/pytorch/blob/main/AI_pytorch_APP_init.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## IMPORT AND MY DEFINED FUNCTIONS

### Imports

In [30]:
import torch
from torch import nn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from torchvision import transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import datasets

from tqdm.auto import tqdm

import sklearn
from sklearn.model_selection import train_test_split

###My functions

In [31]:
## Visualization #####################
########################################################################################
########################################################################################

def details(tensor):
  print("")
  print("Shape: ",tensor.shape)
  print("Dimentions: ",tensor.ndim)
  print("###########")
  print(tensor)
  print("###########")
  print("Tensor Data type: ",tensor.dtype)
  print("Device: ",tensor.device)
  print("")

########################################################################################
def details_plot(train_data, 
                     train_labels, 
                     test_data, 
                     test_labels,
                     predictions=None):
  """
  Plots training data, test data and compares predictions.
  """
  plt.figure(figsize=(10, 7))

  # Plot training data in blue
  plt.scatter(train_data, train_labels, c="b", s=4, label="Training data")
  
  # Plot test data in green
  plt.scatter(test_data, test_labels, c="g", s=4, label="Testing data")

  if predictions is not None:
    # Plot the predictions in red (predictions were made on the test data)
    plt.scatter(test_data, predictions, c="r", s=4, label="Predictions")

  # Show the legend
  plt.legend(prop={"size": 14});


########################################################################################
def details_loss_curve(epochs,train_loss_track,test_loss_track):
  #loss curve
  plt.figure(figsize=(8,8))
  plt.title("Loss curve")
  plt.xlabel("Epochs")
  plt.ylabel("Loss")
  plt.plot(epochs,train_loss_track,label="Train loss")
  plt.plot(epochs,test_loss_track,label="Test loss")
  plt.legend(prop={"size": 14})


In [32]:
## operations ##############################
########################################################################################
########################################################################################

def accu(y_pred,y_test):
  correct=torch.eq(y_pred.squeeze(),y_test.squeeze()).sum().item()
  accuracy=(correct/len(y_test))*100
  return accuracy

def details_plot_scatter(x_train, x_test, y_train, y_test,
                     predictions=None):
  """
  Plots training data, test data and compares predictions.
  """
  plt.figure(figsize=(10, 7))
  plt.scatter(x_train[:,0],x_train[:,1],c=y_train,cmap=plt.cm.BrBG, s=4, label="Training data")
  plt.scatter(x_test[:,0],x_test[:,1],c=y_test, s=60, label="Test data")
  plt.legend(prop={"size": 14});
  if predictions is not None:
    # Plot the predictions in red (predictions were made on the test data)
    plt.scatter(x_test[:,0],x_test[:,1],c=predictions, s=20, label="Prediction")

  # Show the legend
  plt.legend(prop={"size": 14});

########################################################################################

#traning function 
def train(
    train_dataloader,
    model,
    loss_func,
    optimizer,
    accu,
    device):
  from tqdm.auto import tqdm  
  #loss_tracking
  train_loss=0
  train_loss_track=[]
  train_accuracy=0

  #train mode
  model=model.to(device)
  model.train()

  #training loop
  for batch, (X,y) in tqdm(enumerate(train_dataloader)):
    #device set
    X,y=X.to(device),y.to(device)

    

    #forward propegation
    predict=model(X)
    #loss calcilation
    loss=loss_func(predict,y)
    

    train_loss_track.append(loss.detach().item())

    #clear passed gradients
    optimizer.zero_grad()
    #back propogation
    loss.backward()
    #update new values for weight and bias
    optimizer.step()

    #print(loss,batch)
    train_loss_track.append(loss.detach().item())
    
    train_loss+=loss_func(predict,y)
    train_accuracy+=accu(predict.squeeze().argmax(dim=1),y)
  
  #average_loss
  train_loss=loss/len(train_dataloader)
  train_accuracy=train_accuracy/len(train_dataloader)

  print(f"Training loss: {train_loss:.5f} Train_accuracy {train_accuracy:.2f}")

########################################################################################

#testing function

def test(
    test_dataloader,
    model,
    loss_func,
    accu,
    device):
  from tqdm.auto import tqdm  
  #loss_tracking
  test_loss=0
  test_loss_track=[]
  test_accuracy=0

  #testing loop
  model.eval()

  with torch.inference_mode():
    for batch, (X,y) in tqdm(enumerate(test_dataloader)):
      #device set
      X,y=X.to(device),y.to(device)

      
      #forward propegation
      predict=model(X)
      #loss calcilation
      loss=loss_func(predict,y)
      

      test_loss_track.append(loss.detach().item())

      #print(loss,batch)
      test_loss_track.append(loss.detach().item())
      
      test_loss+=loss_func(predict,y)
      test_accuracy+=accu(predict.squeeze().argmax(dim=1),y)
  
  #average_loss
  test_loss=loss/len(test_dataloader)
  test_accuracy=test_accuracy/len(test_dataloader)

  print(f"Testing loss: {test_loss:.5f} Test_accuracy {test_accuracy:.2f}")

########################################################################################

#evaluation function   
def evaluate(model:torch.nn.Module,
             loss_func:torch.nn.Module,
             data_loader:torch.utils.data.DataLoader,
             accu
             ):
  model_name=model.__class__.__name__
  model.eval()
  loss,accuracy=0,0
  with torch.inference_mode():
    for x,y in tqdm(data_loader):
      x,y=x.to(device),y.to(device)
      predict=model(x)
      loss+=loss_func(predict,y)
      accuracy+=accu(predict.squeeze().argmax(dim=1),y)
  loss=loss/len(data_loader)
  accuracy=accuracy/len(data_loader)
  return {"Model Name":model_name,"Accuracy":accuracy,"Loss":loss.item()}



In [33]:
import torch
def details(tensor):
  print("")
  print("Shape: ",tensor.shape)
  print("Dimentions: ",tensor.ndim)
  print("###########")
  print(tensor)
  print("###########")
  print("Tensor Data type: ",tensor.dtype)
  print("Device: ",tensor.device)
  print("")

def details_plot(train_data, 
                     train_labels, 
                     test_data, 
                     test_labels,
                     predictions=None):
  """
  Plots training data, test data and compares predictions.
  """
  plt.figure(figsize=(10, 7))

  # Plot training data in blue
  plt.scatter(train_data, train_labels, c="b", s=4, label="Training data")
  
  # Plot test data in green
  plt.scatter(test_data, test_labels, c="g", s=4, label="Testing data")

  if predictions is not None:
    # Plot the predictions in red (predictions were made on the test data)
    plt.scatter(test_data, predictions, c="r", s=4, label="Predictions")

  # Show the legend
  plt.legend(prop={"size": 14});

def details_loss_curve(epochs,train_loss_track,test_loss_track):
  #loss curve
  plt.figure(figsize=(8,8))
  plt.title("Loss curve")
  plt.xlabel("Epochs")
  plt.ylabel("Loss")
  plt.plot(epochs,train_loss_track,label="Train loss")
  plt.plot(epochs,test_loss_track,label="Test loss")
  plt.legend(prop={"size": 14})

import torch
def details(tensor):
  print("")
  print("Shape: ",tensor.shape)
  print("Dimentions: ",tensor.ndim)
  print("###########")
  print(tensor)
  print("###########")
  print("Tensor Data type: ",tensor.dtype)
  print("Device: ",tensor.device)
  print("")

def details_plot(train_data, 
                     train_labels, 
                     test_data, 
                     test_labels,
                     predictions=None):
  """
  Plots training data, test data and compares predictions.
  """
  plt.figure(figsize=(10, 7))

  # Plot training data in blue
  plt.scatter(train_data, train_labels, c="b", s=4, label="Training data")
  
  # Plot test data in green
  plt.scatter(test_data, test_labels, c="g", s=4, label="Testing data")

  if predictions is not None:
    # Plot the predictions in red (predictions were made on the test data)
    plt.scatter(test_data, predictions, c="r", s=4, label="Predictions")

  # Show the legend
  plt.legend(prop={"size": 14});



def accu(y_pred,y_test):
  correct=torch.eq(y_pred.squeeze(),y_test.squeeze()).sum().item()
  accuracy=(correct/len(y_test))*100
  return accuracy

def details_plot_scatter(x_train, x_test, y_train, y_test,
                     predictions=None):
  """
  Plots training data, test data and compares predictions.
  """
  plt.figure(figsize=(10, 7))
  plt.scatter(x_train[:,0],x_train[:,1],c=y_train,cmap=plt.cm.BrBG, s=4, label="Training data")
  plt.scatter(x_test[:,0],x_test[:,1],c=y_test, s=60, label="Test data")
  plt.legend(prop={"size": 14});
  if predictions is not None:
    # Plot the predictions in red (predictions were made on the test data)
    plt.scatter(x_test[:,0],x_test[:,1],c=predictions, s=20, label="Prediction")

  # Show the legend
  plt.legend(prop={"size": 14});



#traning function #####################################
def train(
    data_loader,
    model,
    loss_func,
    optimizer,
    accu,
    device):
  from tqdm.auto import tqdm  
  #loss_tracking
  train_loss=0
  train_loss_track=[]
  train_accuracy=0

  #train mode
  model=model.to(device)
  model.train()

  #training loop
  for batch, (X,y) in tqdm(enumerate(train_dataloader)):
    #device set
    X,y=X.to(device),y.to(device)

    

    #forward propegation
    predict=model(X)
    #loss calcilation
    loss=loss_func(predict,y)
    

    train_loss_track.append(loss.detach().item())

    #clear passed gradients
    optimizer.zero_grad()
    #back propogation
    loss.backward()
    #update new values for weight and bias
    optimizer.step()

    #print(loss,batch)
    train_loss_track.append(loss.detach().item())
    
    train_loss+=loss_func(predict,y)
    train_accuracy+=accu(predict.squeeze().argmax(dim=1),y)
  
  #average_loss
  train_loss=loss/len(train_dataloader)
  train_accuracy=train_accuracy/len(train_dataloader)

  print(f"Training loss: {train_loss:.5f} Train_accuracy {train_accuracy:.2f}")

  
#testing function##################################
def test(
    data_loader,
    model,
    loss_func,
    accu,
    device):
  from tqdm.auto import tqdm  
  #loss_tracking
  test_loss=0
  test_loss_track=[]
  test_accuracy=0

  #testing loop
  model.eval()

  with torch.inference_mode():
    for batch, (X,y) in tqdm(enumerate(test_dataloader)):
      #device set
      X,y=X.to(device),y.to(device)

      
      #forward propegation
      predict=model(X)
      #loss calcilation
      loss=loss_func(predict,y)
      

      test_loss_track.append(loss.detach().item())

      #print(loss,batch)
      test_loss_track.append(loss.detach().item())
      
      test_loss+=loss_func(predict,y)
      test_accuracy+=accu(predict.squeeze().argmax(dim=1),y)
  
  #average_loss
  test_loss=loss/len(test_dataloader)
  test_accuracy=test_accuracy/len(test_dataloader)

  print(f"Testing loss: {test_loss:.5f} Test_accuracy {test_accuracy:.2f}")

#evaluation function ############################  
def evaluate(model:torch.nn.Module,
             loss_func:torch.nn.Module,
             data_loader:torch.utils.data.DataLoader,
             accu
             ):
  model_name=model.__class__.__name__
  model.eval()
  loss,accuracy=0,0
  with torch.inference_mode():
    for x,y in tqdm(data_loader):
      x,y=x.to(device),y.to(device)
      predict=model(x)
      loss+=loss_func(predict,y)
      accuracy+=accu(predict.squeeze().argmax(dim=1),y)
  loss=loss/len(data_loader)
  accuracy=accuracy/len(data_loader)
  return {"Model Name":model_name,"Accuracy":accuracy,"Loss":loss.item()}



###GPU

In [34]:
!nvidia-smi
print(torch.cuda.is_available())
#setup device agnostic code
device="cuda" if torch.cuda.is_available() else "cpu"
print(device)

/bin/bash: nvidia-smi: command not found
False
cpu


##DATA PROCESS 


ex- tensor data-> x_train,y_train,x_test,y_test


In [35]:
# create data

#weight and bias

  #######to prevent mat multiplication error   .unsqueeze(dim=1)

#dataloader


In [36]:
# create train/test split


In [37]:
# plot and visualize it


##BUILDING THE MODEL


tips
1. torch.nn.find_suitable_loss
2. torch.nn.Paramenter
3. torch.nn.Module this is base class
4. torch.optim - optimizer

In [38]:
#model
class Model_name(nn.Module):
  def __init__(self):
    super().__init__()
    #model_structer
   
  def forward(self,x: torch.Tensor) -> torch.Tensor:


    return ##



#discription of model


In [39]:
#make inference/prediction and visualize the model


##CREATE OPTIMIZER / LOSS_FUNC

In [40]:
#create loss func

#optimizer



##TRAINING / TESTING LOOP

tips
0. Loop throug the data
1. set to training mode
2. Forward pass
3. Calculate the loss(pred to ground truth)
4. optimizer zer grad
5. loss backward .. back propogation
6. optimizer step - use opt and adjust parameter ... grad desent


In [41]:
#initiate trackers and hyper parameters

epochs=10
model="model_name"
 
for epoch in tqdm(range(epochs)):
  print(f"Epoch {epoch}")
  #train(train_dataloader,model,loss_func,optimizer,accu,device)
  #test(test_dataloader,model,loss_func,accu,device)



  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9


##EVALUATION OF THE MODEL

In [42]:
#plot / matrices



##SAVING/LOADING

tips
1.  torch.save() 
2.  torch.load()
3.  toch.nn.Module.load_state_dict()